## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import files


#  Import and read the charity_data.csv.
uploaded = files.upload()


Saving charity_data.csv to charity_data (6).csv


In [2]:
# Convert to Dataframe
import io
filename = list(uploaded.keys())[0]
application_df = pd.read_csv(io.StringIO(uploaded[filename].decode("utf-8")))
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


It looks like "IS_SUCCESSFUL" is going to be the target variable, with the rest, save "EIN" and "NAME" being the featured ones.

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
counts = application_df.nunique()
print(counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts for binning

value_counts = application_df['APPLICATION_TYPE'].value_counts()
print(value_counts)


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE

cutoff = 200

rare_values = value_counts[value_counts < cutoff].index.tolist()

# Bin the "rare" values into the "Other" category
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].apply(lambda x: 'Other' if x in rare_values else x)

# Check if the binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [7]:
# CLASSIFICATION Binning

value_counts_cla = application_df['CLASSIFICATION'].value_counts()

cutoff = 1500

rare_values_cla = value_counts_cla[value_counts_cla < cutoff].index.tolist()


application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].apply(lambda x: 'Other' if x in rare_values_cla else x)

# Check if the binning was successful
print(application_df['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [9]:
#APPLICATION_TYPE    
application_type_dummies = pd.get_dummies(application_df['APPLICATION_TYPE'])
application_df = pd.concat([application_df, application_type_dummies], axis=1)
application_df = application_df.drop('APPLICATION_TYPE', axis=1)

#AFFILIATION  
affiliation_dummies = pd.get_dummies(application_df['AFFILIATION'])
application_df = pd.concat([application_df, affiliation_dummies], axis=1)
application_df = application_df.drop('AFFILIATION', axis=1)

#CLASSIFICATION          
classification_dummies = pd.get_dummies(application_df['CLASSIFICATION'])
application_df = pd.concat([application_df, classification_dummies], axis=1)
application_df = application_df.drop('CLASSIFICATION', axis=1)

#USE_CASE               
use_case_dummies = pd.get_dummies(application_df['USE_CASE'])
application_df = pd.concat([application_df, use_case_dummies], axis=1)
application_df = application_df.drop('USE_CASE', axis=1)

#ORGANIZATION           
organization_dummies = pd.get_dummies(application_df['ORGANIZATION'])
application_df = pd.concat([application_df, organization_dummies], axis=1)
application_df = application_df.drop('ORGANIZATION', axis=1)

#INCOME_AMT              
income_amt_dummies = pd.get_dummies(application_df['INCOME_AMT'])
application_df = pd.concat([application_df, income_amt_dummies], axis=1)
application_df = application_df.drop('INCOME_AMT', axis=1)

#SPECIAL_CONSIDERATIONS  
special_considerations_dummies = pd.get_dummies(application_df['SPECIAL_CONSIDERATIONS'])
application_df = pd.concat([application_df, special_considerations_dummies], axis=1)
application_df = application_df.drop('SPECIAL_CONSIDERATIONS', axis=1)


application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 44 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   STATUS            34299 non-null  int64
 1   ASK_AMT           34299 non-null  int64
 2   IS_SUCCESSFUL     34299 non-null  int64
 3   Other             34299 non-null  uint8
 4   T10               34299 non-null  uint8
 5   T19               34299 non-null  uint8
 6   T3                34299 non-null  uint8
 7   T4                34299 non-null  uint8
 8   T5                34299 non-null  uint8
 9   T6                34299 non-null  uint8
 10  T7                34299 non-null  uint8
 11  T8                34299 non-null  uint8
 12  CompanySponsored  34299 non-null  uint8
 13  Family/Parent     34299 non-null  uint8
 14  Independent       34299 non-null  uint8
 15  National          34299 non-null  uint8
 16  Other             34299 non-null  uint8
 17  Regional          34299 non-nul

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_shape=(X_train.shape[1],)))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 80)                3520      
                                                                 
 dense_17 (Dense)            (None, 32)                2592      
                                                                 
 dense_18 (Dense)            (None, 1)                 33        
                                                                 
Total params: 6,145
Trainable params: 6,145
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5370 - accuracy: 0.7397
Epoch 2/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5367 - accuracy: 0.7393
Epoch 3/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5365 - accuracy: 0.7392
Epoch 4/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5361 - accuracy: 0.7392
Epoch 5/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5365 - accuracy: 0.7391
Epoch 6/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5361 - accuracy: 0.7400
Epoch 7/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5359 - accuracy: 0.7393
Epoch 8/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5363 - accuracy: 0.7399
Epoch 9/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5359 - accuracy: 0.7390
Epoch 10/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5361 - accura

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5764 - accuracy: 0.7262 - 228ms/epoch - 1ms/step
Loss: 0.5764164924621582, Accuracy: 0.7262390851974487


In [27]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')
